# 📐 Geometry & Multi-View Vision

**Topics:** Homography, Epipolar Geometry, Camera Calibration

In [ ]:
# Setup
!pip install opencv-python-headless numpy matplotlib -q
import cv2
import numpy as np
import matplotlib.pyplot as plt
print('✅ Setup complete!')

In [ ]:
# Homography - Perspective Transform
# Define source and destination points
src_pts = np.float32([[0,0], [300,0], [300,200], [0,200]])
dst_pts = np.float32([[50,50], [250,30], [280,180], [20,170]])

# Compute homography matrix
H, _ = cv2.findHomography(src_pts, dst_pts)
print('Homography Matrix H:')
print(H.round(3))

# Create test image
img = np.zeros((200, 300, 3), dtype=np.uint8)
cv2.rectangle(img, (50,50), (250,150), (0,255,0), 2)
cv2.circle(img, (150,100), 30, (255,0,0), -1)

# Apply homography
warped = cv2.warpPerspective(img, H, (300, 200))

fig, axes = plt.subplots(1, 2, figsize=(10, 4))
axes[0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
axes[0].set_title('Original')
axes[1].imshow(cv2.cvtColor(warped, cv2.COLOR_BGR2RGB))
axes[1].set_title('After Homography')
plt.show()

In [ ]:
# Camera Intrinsic Matrix
fx, fy = 500, 500  # Focal length in pixels
cx, cy = 320, 240  # Principal point

K = np.array([
    [fx, 0, cx],
    [0, fy, cy],
    [0, 0, 1]
], dtype=np.float32)

print('Intrinsic Matrix K:')
print(K)

# 3D to 2D projection
P_3d = np.array([[1, 0, -1, 1, 0, -1],
                 [1, 1, 1, -1, -1, -1],
                 [5, 5, 5, 5, 5, 5]], dtype=np.float32)

p_2d = K @ P_3d
p_2d = p_2d[:2] / p_2d[2]

plt.figure(figsize=(6, 5))
plt.scatter(p_2d[0], p_2d[1], c='red', s=100)
plt.xlim(0, 640)
plt.ylim(480, 0)
plt.title('Projected 2D Points')
plt.xlabel('u (pixels)')
plt.ylabel('v (pixels)')
plt.grid(True)
plt.show()

In [ ]:
# Fundamental Matrix & Epipolar Lines
# Given two corresponding point sets
pts1 = np.float32([[100,100], [200,100], [150,200], [100,200], [200,200]])
pts2 = np.float32([[110,105], [210,95], [155,195], [105,205], [205,195]])

# Compute Fundamental Matrix
F, mask = cv2.findFundamentalMat(pts1, pts2, cv2.FM_8POINT)
print('Fundamental Matrix F:')
print(F.round(6))

# Epipolar constraint: x2.T @ F @ x1 = 0
for i in range(len(pts1)):
    x1 = np.append(pts1[i], 1)
    x2 = np.append(pts2[i], 1)
    error = x2 @ F @ x1
    print(f'Point {i}: Epipolar error = {error:.6f}')